In [1]:
import ee
import folium
from folium import Map
from branca.element import Template, MacroElement

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
agro_eco_regions_gee = "users/mtpictd/agro_eco_regions"
roi_boundary = ee.FeatureCollection(agro_eco_regions_gee).filter(ee.Filter.eq('ae_regcode', 1)).first().getInfo()

states = ee.FeatureCollection("projects/ee-indiasat/assets/India_state_boundaries")
districts = ee.FeatureCollection("projects/ee-indiasat/assets/india_district_boundaries")
aezs = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

In [4]:
# Filter to Karnataka (assuming the column is 'state_name' or similar — adjust if needed)
karnataka = states.filter(ee.Filter.eq('Name', 'Karnataka')).first()

In [5]:
# Filter AEZs that intersect with Karnataka
aez_in_karnataka = aezs.filterBounds(karnataka.geometry())

# Print AEZ codes that intersect
aez_list = aez_in_karnataka.aggregate_array('ae_regcode').getInfo()

print("AEZs in Karnataka:", sorted(set(aez_list)))

AEZs in Karnataka: [3, 6, 7, 8, 19]


In [7]:
# Filter Karnataka state and get its geometry
karnataka_geom = states.filter(ee.Filter.eq('Name', 'Karnataka')).geometry()

# Filter districts within Karnataka
karnataka_districts = districts.filterBounds(karnataka_geom)

# Function to map each district to the AEZs it intersects with
def get_district_aezs(district):
    district_geom = district.geometry()
    district_name = district.get('Name')  # change key if needed
    intersecting_aezs = aezs.filterBounds(district_geom)
    aez_codes = intersecting_aezs.aggregate_array('ae_regcode')
    return ee.Feature(None, {
        'district': district_name,
        'aez_codes': aez_codes
    })

# Map over each district
district_aez_mapping = karnataka_districts.map(get_district_aezs)

# Export or print results
results = district_aez_mapping.aggregate_array('district').getInfo()
aez_lists = district_aez_mapping.aggregate_array('aez_codes').getInfo()

for district, aez_list in zip(results, aez_lists):
    print(f"{district}: {sorted(set(aez_list))}")

Anantapur: [3, 7, 8]
Chittoor: [3, 7, 8, 18]
Kurnool: [3, 6, 7]
North Goa: [19]
South Goa: [19]
Bagalkot: [3, 6]
Bangalore Rural: [8]
Bangalore: [8]
Belgaum: [3, 6, 19]
Bellary: [3, 6, 7, 8]
Bidar: [6, 7]
Bijapur: [3, 6]
Chamrajnagar: [8, 19]
Chikballapura: [3, 8]
Chikmagalur: [8, 19]
Chitradurga: [3, 8]
Dakshina Kannada: [19]
Davanagere: [3, 6, 8, 19]
Dharwad: [6]
Gadag: [3, 6]
Gulbarga: [6, 7]
Hassan: [8, 19]
Haveri: [6, 8]
Kodagu: [8, 19]
Kolar: [8]
Koppal: [3, 6]
Mandya: [8]
Mysore: [8, 19]
Raichur: [3, 6, 7]
Ramanagara: [8]
Shimoga: [6, 8, 19]
Tumkur: [3, 8]
Udupi: [19]
Uttara Kannada: [6, 19]
Yadgir: [6, 7]
Kannur: [19]
Kasaragod: [19]
Wayanad: [8, 19]
Kolhapur: [6, 19]
Latur: [6]
Nanded: [6]
Osmanabad: [6]
Sangli: [3, 6, 19]
Sindhudurg: [19]
Solapur: [6]
Dharmapuri: [8]
Erode: [8, 19]
Krishnagiri: [8]
The Nilgiris: [8, 19]
Mahbubnagar: [6, 7]
Medak: [6, 7]
Nizamabad: [6, 7]
Ranga Reddy: [6, 7]


In [1]:
import ee
ee.Initialize()

# Load datasets
states = ee.FeatureCollection("projects/ee-indiasat/assets/India_state_boundaries")
districts = ee.FeatureCollection("projects/ee-indiasat/assets/india_district_boundaries")
aezs = ee.FeatureCollection("users/mtpictd/agro_eco_regions")

# Filter Karnataka
karnataka_geom = states.filter(ee.Filter.eq('Name', 'Karnataka')).geometry()
karnataka_districts = districts.filterBounds(karnataka_geom)

# Function to compute AEZ area percentage in district
def get_aez_percentage(district):
    district_geom = district.geometry()
    district_name = district.get('Name')
    district_area = district_geom.area(1)  # area in square meters

    # Filter AEZs intersecting this district
    intersecting_aezs = aezs.filterBounds(district_geom)

    def compute_percentage(aez):
        aez_code = aez.get('ae_regcode')
        intersection = district_geom.intersection(aez.geometry(), 1)
        intersect_area = intersection.area(1)
        percent = intersect_area.divide(district_area).multiply(100)
        return ee.Feature(None, {
            'district': district_name,
            'aez_code': aez_code,
            'percent_area': percent
        })

    return intersecting_aezs.map(compute_percentage)

# Flatten all per-district AEZ percentage results
aez_percentages = karnataka_districts.map(get_aez_percentage).flatten()

# Export or print results
districts_list = aez_percentages.aggregate_array('district').getInfo()
aez_codes_list = aez_percentages.aggregate_array('aez_code').getInfo()
percent_list = aez_percentages.aggregate_array('percent_area').getInfo()

from collections import defaultdict

result_dict = defaultdict(dict)
for dist, aez, percent in zip(districts_list, aez_codes_list, percent_list):
    result_dict[dist][aez] = round(percent, 2)

# Print results
for district, aez_info in result_dict.items():
    print(f"\n{district}:")
    for aez_code, percent in aez_info.items():
        print(f"  AEZ {aez_code}: {percent}%")



Anantapur:
  AEZ 3: 97.17%
  AEZ 7: 1.73%
  AEZ 8: 1.1%

Chittoor:
  AEZ 3: 0.38%
  AEZ 18: 0.23%
  AEZ 7: 0.3%
  AEZ 8: 99.09%

Kurnool:
  AEZ 3: 1.89%
  AEZ 6: 0.12%
  AEZ 7: 97.98%

North Goa:
  AEZ 19: 99.24%

South Goa:
  AEZ 19: 99.14%

Bagalkot:
  AEZ 3: 94.84%
  AEZ 6: 5.16%

Bangalore Rural:
  AEZ 8: 100%

Bangalore:
  AEZ 8: 100%

Belgaum:
  AEZ 3: 4.06%
  AEZ 6: 90.13%
  AEZ 19: 5.81%

Bellary:
  AEZ 3: 89.0%
  AEZ 6: 10.89%
  AEZ 7: 0.11%
  AEZ 8: 0.0%

Bidar:
  AEZ 6: 99.88%
  AEZ 7: 0.12%

Bijapur:
  AEZ 3: 33.99%
  AEZ 6: 66.01%

Chamrajnagar:
  AEZ 19: 1.4%
  AEZ 8: 98.6%

Chikballapura:
  AEZ 3: 0.81%
  AEZ 8: 99.19%

Chikmagalur:
  AEZ 19: 61.04%
  AEZ 8: 38.96%

Chitradurga:
  AEZ 3: 31.06%
  AEZ 8: 68.94%

Dakshina Kannada:
  AEZ 19: 99.96%

Davanagere:
  AEZ 3: 24.47%
  AEZ 6: 1.53%
  AEZ 19: 0.82%
  AEZ 8: 73.18%

Dharwad:
  AEZ 6: 100%

Gadag:
  AEZ 3: 18.88%
  AEZ 6: 81.12%

Gulbarga:
  AEZ 6: 99.89%
  AEZ 7: 0.11%

Hassan:
  AEZ 19: 23.79%
  AEZ 8: 76.21%

Hav

In [8]:
import ee

# Authenticate and initialize Earth Engine
ee.Initialize()

# Define helper function to add Earth Engine layers to folium
def add_ee_layer(self, ee_object, vis_params, name):
    try:
        if isinstance(ee_object, ee.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        elif isinstance(ee_object, ee.FeatureCollection):
            folium.GeoJson(
                data=ee_object.getInfo(),
                name=name
            ).add_to(self)
    except Exception as e:
        print(f"Error adding layer: {e}")

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer


# Rasterize AEZ codes (use mode reducer in case of overlap)
aez_image = aez_in_karnataka.reduceToImage(['ae_regcode'], ee.Reducer.first())

# Remap AEZ codes to indices
aez_codes = [3, 6, 7, 8, 19]
remapped_indices = list(range(len(aez_codes)))
aez_remapped = aez_image.remap(aez_codes, remapped_indices)

# Define visualization parameters
palette = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00']
vis_params = {
    'min': 0,
    'max': len(remapped_indices) - 1,
    'palette': palette
}

# Initialize Folium map
center = [15.3, 76.5]  # Rough center of Karnataka
m = folium.Map(location=center, zoom_start=7)

# Add AEZ layer to the map
m.add_ee_layer(aez_remapped, vis_params, 'AEZs in Karnataka')

# Add district boundaries
m.add_child(folium.GeoJson(
    data=karnataka_districts.getInfo(),
    name='District Boundaries',
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 1
    }
))

legend_html = """
{% macro html() %}
<div style="
     position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 180px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     &nbsp;<b>AEZ Legend</b><br>
     &nbsp;<i style="background:#e41a1c;width:10px;height:10px;display:inline-block;"></i>&nbsp;AEZ 3<br>
     &nbsp;<i style="background:#377eb8;width:10px;height:10px;display:inline-block;"></i>&nbsp;AEZ 6<br>
     &nbsp;<i style="background:#4daf4a;width:10px;height:10px;display:inline-block;"></i>&nbsp;AEZ 7<br>
     &nbsp;<i style="background:#984ea3;width:10px;height:10px;display:inline-block;"></i>&nbsp;AEZ 8<br>
     &nbsp;<i style="background:#ff7f00;width:10px;height:10px;display:inline-block;"></i>&nbsp;AEZ 19<br>
</div>
{% endmacro %}
"""


legend = MacroElement()
legend._template = Template(legend_html)

# Patch MacroElement to call macro with zero args (no kwargs)
import types
def _render(self, **kwargs):
    return self._template.module.html()
legend.render = types.MethodType(_render, legend)

m.get_root().add_child(legend)

# Display the map
m
